# Inicializar la Dimensión Sede
Importaciones

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import yaml

# Conexión a la base de datos

In [3]:
# Cargar configuración de conexión desde el archivo YAML
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

# Extraer Datos de la Sede

In [4]:
# Query para extraer la información de la dimensión sede
query = """
SELECT
    s.sede_id,
    s.nombre AS sede_nombre,
    s.direccion AS sede_direccion,
    s.telefono AS sede_telefono,
    s.nombre_contacto AS contacto_sede,
    c.nombre AS ciudad_nombre,
    cl.nombre AS cliente_nombre
FROM
    public.sede s
LEFT JOIN
    public.ciudad c ON s.ciudad_id = c.ciudad_id
LEFT JOIN
    public.cliente cl ON s.cliente_id = cl.cliente_id;
"""

# Leer los datos desde la base de datos 'RAPIDOS-Y_FURIOSOS' a un DataFrame de Pandas
df_sede = pd.read_sql(query, ryf_conn)

# Mostrar los primeros registros para verificar la extracción
df_sede.head()

,sede_id,sede_nombre,sede_direccion,sede_telefono,contacto_sede,ciudad_nombre,cliente_nombre
0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
1,11,REMEDIOZ/ 123,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
2,13,DIME / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
3,14,DESPACHOS / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
4,23,POPAYAN BODEGA 28 / A,Los angeles distrito Latino,310-70000,JUAN PEREZ,POPAYAN,CARROS DEL PACIFICO (CHINA)


Cargar la Dimensión Sede

In [5]:
df_sede.to_sql('dim_sede', etl_conn, index=False, if_exists='replace')

52